<a href="https://colab.research.google.com/github/sjtalkar/DP-203-Azure-Data-Engineering-Notes/blob/main/SparkAndDeltaLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!tar xf 'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz' 
!pip -q install findspark

In [12]:
!pwd

/content


In [10]:
%ls 'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz'

'gdrive/My Drive/Databricks/spark-3.0.0-bin-hadoop3.2.tgz'


In [14]:
%ls /content

gdrive/  sample_data/  spark-3.0.0-bin-hadoop3.2/


In [15]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'


In [16]:
import findspark
findspark.init()

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [22]:
from pyspark.sql.types import *

In [25]:
transaction_schema = StructType([
            StructField('msno', StringType()),
            StructField('payment_method_id', IntegerType()),
            StructField('payment_plan_days', IntegerType()),
            StructField('plan_list_price', IntegerType()),
            StructField('actual_amount_paid', IntegerType()),
            StructField('is_auto_renew', IntegerType()),
            StructField('transaction_date', DateType()),
            StructField('membership_expire_date', DateType()),
            StructField('is_cancel', IntegerType()),
            ])


In [31]:
# Work with the data using DELTA

In [26]:
#read data from csv
transactions = (
                spark
                  .read
                  .csv(
                      'gdrive/My Drive/Databricks/transactions.csv',
                       schema=transaction_schema,
                       header=True,
                       dateFormat = 'yyyyMMdd'
                  )
)

In [28]:
#persist in delta lake format - essentially a parquet file with additional features such as history and versioning
# This creates  folders called  G:\My Drive\Databricks\transactions\transaction_date=2015-01-01 ......
(
  transactions
    .write
    .format('delta')
    .partitionBy('transaction_date')
    .mode('overwrite')
    .save('gdrive/My Drive/Databricks/transactions',)    
)

In [32]:
# create database to house SQL tables
_ = spark.sql('CREATE DATABASE IF NOT EXISTS kkbox')

In [33]:
spark.sql("""
    CREATE TABLE kkbox.transactions
    USING DELTA
    LOCATION 'gdrive/My Drive/Databricks/transactions'
""")

DataFrame[]